## Teoría de perturbaciones

Consiste en resolver un sistema perturbado(se conoce la solución al no perturbado), y donde el interés es conocer la contribución de la parte perturbada $H'$ al nuevo sistema total. 

$$ H = H^{0} + H'$$

Para sistemas no degenerados, la corrección a la energía a primer orden se calcula como 

$$ E_{n}^{(1)} = \int\psi_{n}^{(0)*} H' \psi_{n}^{(0)}d\tau$$

** Tarea 1 : Programar esta ecuación si conoces $H^{0}$ y sus soluciones. **

Y la corrección a la función de onda, también a primer orden, se obtiene como:

 $$ \psi_{n}^{(1)} = \sum_{m\neq n} \frac{\langle\psi_{m}^{(0)} | H' | \psi_{n}^{(0)} \rangle}{E_{n}^{(0)} - E_{m}^{(0)}} \psi_{m}^{(0)}$$

**Tarea 2: Programar esta ecuación si conoces $H^{0}$ y sus soluciones. **

**Tarea 3: Investigue las soluciones a segundo orden y también programe las soluciones. **

**Tarea 4.  Resolver el átomo de helio aplicando los programas anteriores.** 

**Tarea 5: Método variacional-perturbativo. **

Este método nos permite estimar de forma precisa $E^{(2)}$ y correcciones perturbativas de la energía de órdenes más elevados para el estado fundamental del sistema, sin evaluar sumas infinitas. Ver ecuación 9.38 del libro. 

**Resolver el átomo de helio, considerando este método (sección 9.4), como mejor le parezca. **

**Tarea 6. Revisar sección 9.7. **

Inicialmente a mano, y sengunda instancia favor de intentar programar sección del problema, i.e. integral de Coulomb  e integral de intercambio.

In [2]:
from sympy import *
from sympy import init_printing; init_printing(use_latex = 'mathjax')
import numpy as np
import sympy.physics.hydrogen as hyd
import mpmath as mp
import sympy.functions.special.spherical_harmonics as sp
import scipy.integrate as integ
import scipy.constants as scc

In [3]:
var('x r r_1 r_2 phi_1 phi_2 theta_1 theta_2 phi theta r_r zeta')
var('n l m', positive = 'True', integer = 'True')
var('m_e hbar Z omega a_0 e L c d Z', real = 'True', constant = 'True')

(mₑ, h̅, Z, ω, a₀, e, L, c, d, Z)

### Átomo de hidrógeno

In [4]:
Yhyd = hyd.R_nl(n, l, r)
Yhyd

          -r                                                            
       l  ───                                                           
  ⎛2⋅r⎞    n                ⎛                     2⋅r⎞   _______________
2⋅⎜───⎟ ⋅ℯ   ⋅assoc_laguerre⎜-l + n - 1, 2⋅l + 1, ───⎟⋅╲╱ (-l + n - 1)! 
  ⎝ n ⎠                     ⎝                      n ⎠                  
────────────────────────────────────────────────────────────────────────
                             2   __________                             
                            n ⋅╲╱ (l + n)!                              

In [5]:
Ehyd = hyd.E_nl(n)
Ehyd

-1  
────
   2
2⋅n 

In [6]:
#Corrección energía primer orden

In [7]:
Ehyd_1 = integrate(Yhyd.subs([(n, 1), (l, 0)])*(hbar/I)**4*(-1/(8*m**3*c**2))*\
          diff(Yhyd.subs([(n, 1), (l, 0)]), r, 4)*r**2, (r, 0, oo))
Ehyd_1

    4  
 -h̅   
───────
   2  3
8⋅c ⋅m 

In [8]:
Ehyd_1.subs([(hbar,scc.hbar),(c, scc.c),(m,scc.m_e)]) /(4*scc.epsilon_0)**4

-1.44634324025469e-22

In [9]:
#Correción función primer orden

In [10]:
Yhyd_num = [simplify(integrate(Yhyd.subs([(n, i), (l, 0)])*(hbar/I)**4*(-1/(8*m**3*c**2))*\
          diff(Yhyd.subs([(n, 1), (l, 0)]), r, 4)*sin(theta)*r**2,\
                (r, 0, 1), (theta, 0, pi), (phi, 0, 2*pi)).args[0][0]) for i in range(2,5)]
Yhyd_num

⎡          4              4            4 ⎤
⎢-5⋅√2⋅π⋅h̅    -49⋅√3⋅π⋅h̅    -979⋅π⋅h̅  ⎥
⎢────────────, ─────────────, ───────────⎥
⎢      2  3            2  3          2  3⎥
⎣  48⋅c ⋅m       1215⋅c ⋅m    23040⋅c ⋅m ⎦

In [11]:
Yhyd_den = [(Ehyd.subs([(n, 1)]) - Ehyd.subs([(n, i)])) for i in range(2, 5)]
Yhyd_den

⎡            -15 ⎤
⎢-3/8, -4/9, ────⎥
⎣             32 ⎦

In [12]:
Yhyd_1 = sum([(Yhyd_num[i]*Yhyd.subs([(n, i+1), (l, 0)]))/Yhyd_den[i] for i in range(3)])
Yhyd_1

                                                           -r                 
                      -r                 ⎛   2          ⎞  ───                
                      ───              4 ⎜2⋅r           ⎟   3                 
          4            2    979⋅√3⋅π⋅h̅ ⋅⎜──── - 2⋅r + 3⎟⋅ℯ               4  -
49⋅√6⋅π⋅h̅ ⋅(-r + 2)⋅ℯ                   ⎝ 9            ⎠        5⋅√2⋅π⋅h̅ ⋅ℯ 
───────────────────────── + ────────────────────────────────── + ─────────────
              2  3                             2  3                    2  3   
        2160⋅c ⋅m                      145800⋅c ⋅m                  9⋅c ⋅m    

 
 
 
r
 
─
 
 

In [13]:
#Corrección energía segundo orden

In [14]:
Ehyd_2_num = [simplify(integrate(Yhyd.subs([(n, i), (l, 0)])*(-1/(8*m**3*c**2))*\
          diff(Yhyd.subs([(n, 1), (l, 0)])*sin(theta)*r**2, r, 4),\
                (r, 0, 1), (theta, 0, pi), (phi, 0, 2*pi)).args[0][0]**2) for i in range(2,5)]
Ehyd_2_num

⎡        2             2                2 ⎤
⎢ 24649⋅π     2719201⋅π     1159879249⋅π  ⎥
⎢──────────, ────────────, ───────────────⎥
⎢      4  6          4  6             4  6⎥
⎣1152⋅c ⋅m   492075⋅c ⋅m   530841600⋅c ⋅m ⎦

In [15]:
Ehyd_2_den = [(Ehyd.subs([(n, 1)]) - Ehyd.subs([(n, i)])) for i in range(2, 5)]
Ehyd_2_den

⎡            -15 ⎤
⎢-3/8, -4/9, ────⎥
⎣             32 ⎦

In [16]:
Ehyd_2 = sum([Ehyd_2_num[i] / Ehyd_2_den[i] for i in range(3)])
Ehyd_2 *(27.12*scc.eV)

                       2 
-3.22200858259767e-16⋅π  
─────────────────────────
           4  6          
          c ⋅m           

### Oscilador armónico

In [17]:
import sympy.physics.qho_1d as qho

In [18]:
Yqho = qho.psi_n(n, x, m, omega)
Yqho

                        2                     
 -n               -m⋅ω⋅x                      
 ───              ────────                    
  2  4 ___ 4 ___    2⋅ℏ           ⎛   √m⋅√ω⋅x⎞
2   ⋅╲╱ m ⋅╲╱ ω ⋅ℯ        ⋅hermite⎜n, ───────⎟
                                  ⎝      √ℏ  ⎠
──────────────────────────────────────────────
              4 ___ 4 ___   ____              
              ╲╱ ℏ ⋅╲╱ π ⋅╲╱ n!               

In [19]:
Eqho = qho.E_n(n, omega)
Eqho

ℏ⋅ω⋅(n + 1/2)

In [20]:
#Corrección energía primer orden

In [21]:
H_1 = c*x**3 + d*x**4

In [22]:
Eqho_1 = integrate(Yqho.subs([(n, 0)])*H_1*Yqho.subs([(n, 0)]), (x, -oo, oo))
Eqho_1.args[0][0]

    2  
 3⋅ℏ ⋅d
───────
   2  2
4⋅m ⋅ω 

In [23]:
#Corrección función primer orden

In [24]:
Yqho_num = [integrate(Yqho.subs([(n, i)])*H_1*Yqho.subs([(n, 0)]), (x, -oo, oo)) for i in range(4) if i != 0]
Yqho_num = [Yqho_num[i].args[0][0] for i in range(3)]
Yqho_num

⎡      3/2          2         3/2   ⎤
⎢3⋅√2⋅ℏ   ⋅c  3⋅√2⋅ℏ ⋅d   √3⋅ℏ   ⋅c ⎥
⎢───────────, ─────────, ───────────⎥
⎢   3/2  3/2      2  2      3/2  3/2⎥
⎣4⋅m   ⋅ω      2⋅m ⋅ω    2⋅m   ⋅ω   ⎦

In [25]:
Yqho_den = [(Eqho.subs([(n, 0)]) - Eqho.subs([(n, i)])) for i in range(4) if i != 0]
Yqho_den

[-ℏ⋅ω, -2⋅ℏ⋅ω, -3⋅ℏ⋅ω]

In [26]:
Yqho_1 = sum([(Yqho_num[i]*Yqho.subs([(n, i+1)]))/Yqho_den[i] for i in range(3)])
simplify(Yqho_1)

                                                          2 
                                                    -m⋅ω⋅x  
⎛     2  2  3                          2      2  ⎞  ────────
⎜  c⋅m ⋅ω ⋅x                3⋅ℏ⋅d⋅m⋅ω⋅x    3⋅ℏ ⋅d⎟    2⋅ℏ   
⎜- ────────── - ℏ⋅c⋅m⋅ω⋅x - ──────────── + ──────⎟⋅ℯ        
⎝      3                         2           4   ⎠          
────────────────────────────────────────────────────────────
                    5/4 4 ___  7/4  11/4                    
                   ℏ   ⋅╲╱ π ⋅m   ⋅ω                        

In [27]:
#Corrección energía segundo orden

In [28]:
Eqho_2_num = [integrate(Yqho.subs([(n, i)])*H_1*Yqho.subs([(n, 0)]), (x, -oo, oo)) for i in range(4) if i != 0]
Eqho_2_num = [Eqho_2_num[i].args[0][0]**2 for i in range(3)]
Eqho_2_num

⎡   3  2     4  2     3  2⎤
⎢9⋅ℏ ⋅c   9⋅ℏ ⋅d   3⋅ℏ ⋅c ⎥
⎢───────, ───────, ───────⎥
⎢   3  3     4  4     3  3⎥
⎣8⋅m ⋅ω   2⋅m ⋅ω   4⋅m ⋅ω ⎦

In [29]:
Eqho_2_den = [(Eqho.subs([(n, 0)]) - Eqho.subs([(n, i)])) for i in range(4) if i != 0]
Eqho_2_den

[-ℏ⋅ω, -2⋅ℏ⋅ω, -3⋅ℏ⋅ω]

In [30]:
Eqho_2 = sum([Eqho_2_num[i]/Eqho_2_den[i] for i in range(3)])
Eqho_2

      2  2      3  2
  11⋅ℏ ⋅c    9⋅ℏ ⋅d 
- ──────── - ───────
     3  4       4  5
  8⋅m ⋅ω     4⋅m ⋅ω 

### Partícula en caja

In [31]:
Ypiab = sqrt(2/L) * sin(n*pi * x/L)
Ypiabm = Ypiab.subs(n,m)
Ypiab

       ___           
      ╱ 1     ⎛π⋅n⋅x⎞
√2⋅  ╱  ─ ⋅sin⎜─────⎟
   ╲╱   L     ⎝  L  ⎠

In [32]:
Epiab = (hbar*pi*n)**2/(2*m_e*L**2)
Epiab

 2   2  2
π ⋅h̅ ⋅n 
─────────
    2    
 2⋅L ⋅mₑ 

In [33]:
#Correción energía primer orden

In [34]:
H_1 = x**2
Epiab_1 = integrate(Ypiab.subs([(n, 1)])*H_1*Ypiab.subs([(n, 1)]), (x, 0, L)).simplify()
Epiab_1

    2     2
   L     L 
- ──── + ──
     2   3 
  2⋅π      

In [35]:
#Correción función de onda primer orden

In [36]:
def ev(x, eL, en, em, epi, H):
    return Ypiab.subs([(L, eL), (n, en), (pi, epi)])*H*Ypiabm.subs([(L, eL), (m, em), (pi, epi)])

In [37]:
en = 1
eL = 1e-9
Ypiab_num = np.array([integ.quad(lambdify(x, ev(x, eL, en, i, np.pi, H_1), 'numpy'), 0, eL)[0] for i in range(1,5) if i != en])
Ypiab_den = [Epiab.subs([(n, en), (L, eL)]) - Epiab.subs([(n, i), (L, eL)]) for i in range(1,5) if i != en]

In [133]:
Ampiab = Ypiab_num / Ypiab_den
Ampiab[0], Ampiab[1], Ampiab[2]

⎛1.20084365798327e-37⋅mₑ  -9.49886096646936e-39⋅mₑ   1.92134985277342e-39⋅mₑ⎞
⎜───────────────────────, ─────────────────────────, ───────────────────────⎟
⎜          2   2                     2   2                     2   2        ⎟
⎝         π ⋅h̅                     π ⋅h̅                     π ⋅h̅         ⎠

In [39]:
Ypiab_1 = sum([Ampiab[i-2] * Ypiab.subs([(n, i), (L, eL)]) for i in range(1,5) if i != 1])
Ypiab_1

3.79740107299537e-33⋅√2⋅mₑ⋅sin(2000000000.0⋅π⋅x)   3.00380358313115e-34⋅√2⋅mₑ⋅
──────────────────────────────────────────────── - ───────────────────────────
                      2   2                                              2   2
                     π ⋅h̅                                              π ⋅h̅ 

sin(3000000000.0⋅π⋅x)   6.07584171679318e-35⋅√2⋅mₑ⋅sin(4000000000.0⋅π⋅x)
───────────────────── + ────────────────────────────────────────────────
                                              2   2                     
                                             π ⋅h̅                      

In [40]:
#Corrección energía segundo orden

In [41]:
Epiab_2_num = np.array([integ.quad(lambdify(x, ev(x, eL, en, i, np.pi, H_1), 'numpy'), 0, eL)[0]**2 for i in range(1,5) if i != en])
Epiab_2_num

array([  3.24455735e-38,   1.44365375e-39,   2.07651671e-40])

In [42]:
Epiab_2_den = [Epiab.subs([(n, en), (L, eL)]) - Epiab.subs([(n, i), (L, eL)]) for i in range(1,5) if i != en]
Epiab_2_den

⎡          2   2             2   2             2   2 ⎤
⎢-1.5e+18⋅π ⋅h̅    -4.0e+18⋅π ⋅h̅    -7.5e+18⋅π ⋅h̅  ⎥
⎢────────────────, ────────────────, ────────────────⎥
⎣       mₑ                mₑ                mₑ       ⎦

In [43]:
Epiab_2 = sum([Epiab_2_num[i]/Epiab_2_den[i] for i in range(3)])
Epiab_2

-2.20189826918465e-56⋅mₑ 
─────────────────────────
           2   2         
          π ⋅h̅          

### Átomo de helio

In [44]:
#Corrección energía primer orden

In [45]:
Yhe_1 = hyd.R_nl(n, l, r_1, Z = 2)
Yhe_2 = Yhe_1.subs([(r_1, r_2)])
Yhe_1

              -2⋅r₁                                                           
           l  ──────                                                          
     ⎛4⋅r₁⎞     n                  ⎛                     4⋅r₁⎞   _____________
4⋅√2⋅⎜────⎟ ⋅ℯ      ⋅assoc_laguerre⎜-l + n - 1, 2⋅l + 1, ────⎟⋅╲╱ (-l + n - 1)
     ⎝ n  ⎠                        ⎝                      n  ⎠                
──────────────────────────────────────────────────────────────────────────────
                                 2   __________                               
                                n ⋅╲╱ (l + n)!                                

  
  
__
! 
  
──
  
  

In [46]:
Ehe = hyd.E_nl(n, Z=2)
Ehe

-2 
───
  2
 n 

In [47]:
def Ehe_r(nn, mm):
    return Yhe_1.subs([(n, nn), (l, 0)])*Yhe_2.subs([(n, mm), (l, 0)])*r_1**2*r_2**2*(1/r_r)
Ehe_r(1, 1)

     2   2  -2⋅r₁  -2⋅r₂
32⋅r₁ ⋅r₂ ⋅ℯ     ⋅ℯ     
────────────────────────
           rᵣ           

In [48]:
def Ehe_ev(nn, mm):
    return integrate(integrate(Ehe_r(1, nn).subs([(r_r, r_2)]), (r_1, 0, r_2)) + \
                 integrate(Ehe_r(1, mm).subs([(r_r, r_1)]), (r_1, r_2, oo)), (r_2, 0, oo))

print(Ehe_ev(1, 1) * (27.2114), 'eV' )

34.0142500000000 eV


## Variacional-perturbativo

In [50]:
Yhe_vp = hyd.R_nl(1, 0, r_1, Z=zeta) * hyd.R_nl(1, 0, r_2, Z=zeta)
simplify(Yhe_vp)

   3  -ζ⋅(r₁ + r₂)
4⋅ζ ⋅ℯ            

In [110]:
Ehe_vp = integrate(integrate(Yhe_vp*(zeta - Z)*((1/r_1)+(1/r_2))*Yhe_vp*r_1**2*r_2**2, (r_1, 0, oo)).args[0][0]\
                   , (r_2, 0, oo)).args[0][0] - zeta**2
Ehe_vp

          2
-2⋅Z⋅ζ + ζ 

In [111]:
Ehe_vp_1 = Ehe_vp + Ehe_ev(1, 1)*zeta/2
Ehe_vp_1

          2   5⋅ζ
-2⋅Z⋅ζ + ζ  + ───
               8 

In [114]:
Ehe_vp_deriv = diff(Ehe_vp_1, zeta, 1)
Ehe_vp_deriv

-2⋅Z + 2⋅ζ + 5/8

In [121]:
zeta_min = solve(Ehe_vp_deriv, zeta)[0]
zeta_min

Z - 5/16

In [124]:
Ehe_vp_corr = simplify(Ehe_vp_1.subs([(zeta, zeta_min)]))
Ehe_vp_corr

   2   5⋅Z    25
- Z  + ─── - ───
        8    256

In [130]:
print(Ehe_vp_corr.subs([(Z, 2)]) * (27.2114), 'eV')

-77.4887132812500 eV
